## Clustering Crypto

In [ ]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [ ]:
# Load the crypto_data.csv dataset.
file_path = "/Users/aitorgoyenechea/Desktop/Uoft/Gitcode/Cryptocurrencies/Resources/crypto_data.csv"
df_crypto = pd.read_csv(file_path,index_col = 0)
df_crypto.head()

In [ ]:
# Keep all the cryptocurrencies that are being traded.
traded_df = df_crypto[df_crypto.IsTrading == True]
traded_df.head(10)

In [ ]:
# Keep all the cryptocurrencies that have a working algorithm.
working_df = traded_df.dropna(axis=0, subset=["Algorithm"])
working_df.head(10)

In [ ]:
# Remove the "IsTrading" column. 
new_crypto = working_df.drop("IsTrading", axis=1)
new_crypto.head(10)

In [ ]:
# Remove rows that have at least 1 null value.
new_crypto.dropna(inplace=True)
new_crypto.head(10)

In [ ]:
newer_crypto = new_crypto[new_crypto.TotalCoinsMined > 0]
newer_crypto.head(10)

In [ ]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_name_df = pd.DataFrame(newer_crypto["CoinName"])
coin_name_df.head()

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = newer_crypto.drop(["CoinName"], axis=1)
crypto_df.head(10)

In [ ]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
X.head(10)

In [ ]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

# Deliverable 2: Reducing Data Dimensions Using PCA

In [ ]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca

In [ ]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data=X_pca,
    columns=['PC 1', 'PC 2', 'PC 3'],
    index=X.index
)
pcs_df.head(10)

# Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [ ]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

In [ ]:
# Initialize the K-means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

predictions

In [ ]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = coin_name_df["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

# Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name = "CoinName",
    hover_data = ["Algorithm"],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply', 'Class'])

In [ ]:
# Print the total number of tradable cryptocurrencies.
print(f' There are {clustered_df["CoinName"].count()} tradable cryptocurrencies')

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaling_data = clustered_df.copy()
scaling_data_2 = scaling_data.drop(
    ["Algorithm", "ProofType","PC 1", "PC 2", "PC 3", "CoinName", "Class"], 
    axis=1
)
min_max = MinMaxScaler().fit_transform(scaling_data_2)
min_max

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# Create DF for new data
min_max_df = pd.DataFrame(
    data=min_max,
    columns=["TotalCoinsMined", "TotalCoinSupply"],
    index=clustered_df.index
)

# Reorder columns
plot_df = min_max_df.reindex(columns=["TotalCoinSupply","TotalCoinsMined"])

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = coin_name_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = clustered_df["Class"]

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", 
                       y="TotalCoinSupply", 
                       hover_cols=["CoinName"], 
                       by="Class")
